In [ ]:
import  digitalhub as dh

project = dh.get_or_create_project("faudit-classifier-demo")

## 1. Create and Preprocess dataset

In [ ]:
try:
    func = project.get_function("create-dataset")
except:
    func = project.new_function(
        name="create-dataset", 
        kind="python", 
        python_version="PYTHON3_10", 
        code_src="src/fine_tuning_seq2seq.py",  
        handler="preprocess_dataset",
        requirements=["datasets[audio]==3.6.0", "transformers==4.56.1", "torch==2.8.0", "accelerate==1.10.1", "evaluate==0.4.5", "jiwer==4.0.0"]
    )

In [ ]:
train_run = func.run(action="job",
                     parameters={
                         "model_id": "openai/whisper-small",
                         "artifact_name": "audio-dataset",
                         "dataset_name": "fsicoli/common_voice_17_0",
                         "language": "Italian",
                         "language_code": "it",
                         "max_train_samples": 100,
                         "max_eval_samples": 100
                     },
                     secrets=["HF_TOKEN"],
                     envs=[
                        {"name": "HF_HOME", "value": "/local_data/huggingface"},
                        {"name": "TRANSFORMERS_CACHE", "value":  "/local_data/huggingface"}
                     ],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-llmpa",
                        "mount_path": "/local_data",
                        "spec": { "size": "300Gi" }}]
					)

## 2. Fine-tuning

In [ ]:
try:
    func = project.get_function("train-whisper")
except:
    func = project.new_function(
        name="train-whisper", 
        kind="python", 
        python_version="PYTHON3_10", 
        code_src="src/fine_tuning_seq2seq.py",  
        handler="train_and_log_model",
        requirements=["datasets[audio]==3.6.0", "transformers==4.52.0", "torch==2.8.0", "accelerate==1.10.1", "evaluate==0.4.5", "jiwer==4.0.0"]
    )
    build_run = func.run(action="build")

In [ ]:
train_run = func.run(action="job",
                     parameters={
                         "model_id": "openai/whisper-small",
                         "model_name": "whisper-ft",
                         "dataset_name": "fsicoli/common_voice_17_0",
                         "language": "Italian",
                         "language_code": "it",
                         "max_train_samples": 100,
                         "max_eval_samples": 100,
                         "eval_steps": 100,
                         "save_steps": 100,
                         "max_steps": 500,
                         "warmup_steps": 50
                     },
                     profile="1xa100",
                     secrets=["HF_TOKEN"],
                     envs=[
                        {"name": "HF_HOME", "value": "/local_data/huggingface"},
                        {"name": "TRANSFORMERS_CACHE", "value":  "/local_data/huggingface"}
                     ],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-llmpa",
                        "mount_path": "/local_data",
                        "spec": { "size": "100Gi" }}]
					)

## 3. Convert to FasterWhisper
download model, convert to FasterWhisper and test.

In [ ]:
model = project.get_model("whisper-ft")
model.download("./model/whisper-ft", overwrite=True)

In [ ]:
%pip install faster-whisper transformers torch==2.8.0

In [ ]:
!wget -O audio.wav https://github.com/user-attachments/assets/711d1279-6af9-4c6c-a052-e59e7730b757

In [ ]:
from ctranslate2.converters import TransformersConverter

tc = TransformersConverter("./model/whisper-ft", copy_files=['tokenizer.json', 'preprocessor_config.json', 'README.md'])
tc.convert('./model/faster-whisper-ft', quantization="float16")

In [ ]:
from faster_whisper import WhisperModel

model = WhisperModel('./model/faster-whisper-ft', device="cpu")


In [ ]:
segments, info = model.transcribe("audio.wav", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))